In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/combinedgbmp/combined.csv
/kaggle/input/finaldataset/final_dataset_rp.csv
/kaggle/input/gujaratif/final_gujarati (1).csv


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import torch

# Step 1: Load CSV dataset
dataset_path = '//kaggle/input/finaldataset/final_dataset_rp.csv'
df = pd.read_csv(dataset_path)

# Step 2: Preprocess the Data
tokenizer = AutoTokenizer.from_pretrained('ai4bharat/indic-bert')
max_length = 128  # You can adjust this based on your data and model requirements

class CustomDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = tokenizer(text, truncation=True, padding='max_length', max_length=max_length, return_tensors='pt')
        return {'input_ids': encoding['input_ids'].squeeze(), 'attention_mask': encoding['attention_mask'].squeeze(), 'labels': torch.tensor(label)}

# Convert labels to numerical format
label_mapping = {label: idx for idx, label in enumerate(df['result'].unique())}
df['result'] = df['result'].map(label_mapping)

# Split the dataset
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

train_dataset = CustomDataset(train_df['text'].tolist(), train_df['result'].tolist())
test_dataset = CustomDataset(test_df['text'].tolist(), test_df['result'].tolist())


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


config.json:   0%|          | 0.00/507 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/5.65M [00:00<?, ?B/s]

In [5]:
from sklearn.metrics import accuracy_score

tokenizer = AutoTokenizer.from_pretrained('ai4bharat/indic-bert')

class SarcasmDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )

        input_ids = encoding['input_ids'].flatten()
        attention_mask = encoding['attention_mask'].flatten()

        return {
            'text': text,
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Convert labels to numerical format
label_mapping = {label: idx for idx, label in enumerate(df['result'].unique())}
df['result'] = df['result'].map(label_mapping)

# Split the dataset
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

train_dataset = SarcasmDataset(train_df['text'].tolist(), train_df['result'].tolist(), tokenizer, max_len=128)
val_dataset = SarcasmDataset(val_df['text'].tolist(), val_df['result'].tolist(), tokenizer, max_len=128)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

# Step 4: Fine-tuning
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForSequenceClassification.from_pretrained('ai4bharat/indic-bert', num_labels=len(label_mapping))
model.to(device)

optimizer = AdamW(model.parameters(), lr=5e-5)

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    train_true_labels = []
    train_pred_labels = []

    for batch in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs}'):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        
        # Record true labels and predicted labels for accuracy calculation
        train_true_labels.extend(labels.cpu().numpy())
        train_pred_labels.extend(torch.argmax(outputs.logits, dim=1).cpu().numpy())

        total_loss += loss.item()

    # Calculate accuracy for the epoch
    epoch_accuracy = accuracy_score(train_true_labels, train_pred_labels)

    # Print average loss and accuracy for the epoch
    average_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{num_epochs}, Average Training Loss: {average_loss:.4f}, Training Accuracy: {epoch_accuracy * 100:.2f}%")

# Step 5: Evaluation
model.eval()
all_labels = []
all_predictions = []

with torch.no_grad():
    for batch in tqdm(val_loader, desc='Evaluating'):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=1).cpu().numpy()

        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predictions)

# Calculate accuracy
accuracy = accuracy_score(all_labels, all_predictions)
print(f"Validation Accuracy: {accuracy * 100:.2f}%")


Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at ai4bharat/indic-bert and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1/10: 100%|██████████| 1600/1600 [04:20<00:00,  6.15it/s]


Epoch 1/10, Average Training Loss: 0.2356, Training Accuracy: 90.94%


Epoch 2/10: 100%|██████████| 1600/1600 [04:19<00:00,  6.16it/s]


Epoch 2/10, Average Training Loss: 0.1159, Training Accuracy: 95.99%


Epoch 3/10: 100%|██████████| 1600/1600 [04:19<00:00,  6.16it/s]


...

Epoch 9/10, Average Training Loss: 0.0508, Training Accuracy: 98.48%


Epoch 10/10: 100%|██████████| 1600/1600 [04:17<00:00,  6.21it/s]


Epoch 10/10, Average Training Loss: 0.0337, Training Accuracy: 98.95%


Evaluating: 100%|██████████| 400/400 [00:22<00:00, 17.97it/s]

Validation Accuracy: 97.44%
